

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/ER_ICDO.ipynb)




# **ICDO coding**

To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload `license_keys.json` to the folder that opens.
Otherwise, you can look at the example outputs at the bottom of the notebook.



## 1. Colab Setup

Import license keys

In [1]:
import os
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

sparknlp_version = license_keys["PUBLIC_VERSION"]
jsl_version = license_keys["JSL_VERSION"]

print ('SparkNLP Version:', sparknlp_version)
print ('SparkNLP-JSL Version:', jsl_version)

Saving v3_spark_nlp_for_healthcare.json to v3_spark_nlp_for_healthcare.json
SparkNLP Version: 3.0.1
SparkNLP-JSL Version: 3.0.0


Install dependencies

In [2]:
%%capture
for k,v in license_keys.items(): 
    %set_env $k=$v

!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
!bash jsl_colab_setup.sh

Import dependencies into Python

In [3]:
import pandas as pd
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import sparknlp
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl


Start the Spark session

In [4]:
spark = sparknlp_jsl.start(license_keys['SECRET'])

# manually start session
# params = {"spark.driver.memory" : "16G",
#           "spark.kryoserializer.buffer.max" : "2000M",
#           "spark.driver.maxResultSize" : "2000M"}

# spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

## 2. Select the Entity Resolver model and construct the pipeline

Select the models:

**ICDO Entity Resolver models:**

1.   **chunkresolve_icdo_clinical**

**NER models that support neoplasms:**
1.   **ner_bionlp**

For more details: https://github.com/JohnSnowLabs/spark-nlp-models#pretrained-models---spark-nlp-for-healthcare

In [5]:
# Change this to the model you want to use and re-run the cells below.
ER_MODEL_NAME = "chunkresolve_icdo_clinical"
NER_MODEL_NAME = "ner_bionlp"

Create the pipeline

In [6]:
document_assembler = DocumentAssembler() \
    .setInputCol('text')\
    .setOutputCol('document')

sentence_detector = SentenceDetector() \
    .setInputCols(['document'])\
    .setOutputCol('sentences')

tokenizer = Tokenizer()\
    .setInputCols(['sentences']) \
    .setOutputCol('tokens')

embeddings = WordEmbeddingsModel.pretrained('embeddings_clinical', 'en', 'clinical/models')\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

clinical_ner_model = MedicalNerModel.pretrained(NER_MODEL_NAME, "en", "clinical/models") \
    .setInputCols(["sentences", "tokens", "embeddings"])\
    .setOutputCol("clinical_ner_tags")

# using whitelist to filter out entities
clinical_ner_chunker = NerConverter()\
    .setInputCols(["sentences", "tokens", "clinical_ner_tags"])\
    .setOutputCol("ner_chunk").setWhiteList(['Cancer'])

chunk_embeddings = ChunkEmbeddings()\
    .setInputCols("ner_chunk", "embeddings")\
    .setOutputCol("chunk_embeddings")

entity_resolver = \
    ChunkEntityResolverModel.pretrained(ER_MODEL_NAME,"en","clinical/models")\
    .setInputCols("tokens","chunk_embeddings").setOutputCol("resolution")

pipeline = Pipeline(stages=[
    document_assembler, 
    sentence_detector,
    tokenizer,
    embeddings,
    clinical_ner_model,
    clinical_ner_chunker,
    chunk_embeddings,
    entity_resolver])

empty_df = spark.createDataFrame([['']]).toDF("text")
pipeline_model = pipeline.fit(empty_df)
light_pipeline = LightPipeline(pipeline_model)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_bionlp download started this may take some time.
Approximate size to download 13.9 MB
[OK!]
chunkresolve_icdo_clinical download started this may take some time.
Approximate size to download 8.2 MB
[OK!]


## 3. Create example inputs

In [7]:
# Enter examples as strings in this array
input_list = [
"""DIAGNOSIS: Left breast adenocarcinoma stage T3 N1b M0, stage IIIA.
She has been found more recently to have stage IV disease with metastatic deposits and recurrence involving the chest wall and lower left neck lymph nodes.
PHYSICAL EXAMINATION
NECK: On physical examination palpable lymphadenopathy is present in the left lower neck and supraclavicular area. No other cervical lymphadenopathy or supraclavicular lymphadenopathy is present.
RESPIRATORY: Good air entry bilaterally. Examination of the chest wall reveals a small lesion where the chest wall recurrence was resected. No lumps, bumps or evidence of disease involving the right breast is present.
ABDOMEN: Normal bowel sounds, no hepatomegaly. No tenderness on deep palpation. She has just started her last cycle of chemotherapy today, and she wishes to visit her daughter in Brooklyn, New York. After this she will return in approximately 3 to 4 weeks and begin her radiotherapy treatment at that time.""",
    ]

# 4. Run the pipeline

In [8]:
df = spark.createDataFrame(pd.DataFrame({"text": input_list}))
result = pipeline_model.transform(df)
light_result = light_pipeline.fullAnnotate(input_list[0])

# 5. Visualize

Full Pipeline

In [9]:
result.select(
    F.explode(
        F.arrays_zip('ner_chunk.result', 
                     'ner_chunk.begin',
                     'ner_chunk.end',
                     'ner_chunk.metadata',
                     'resolution.metadata', 'resolution.result')
    ).alias('cols')
).select(
    F.expr("cols['0']").alias('chunk'),
    F.expr("cols['1']").alias('begin'),
    F.expr("cols['2']").alias('end'),
    F.expr("cols['3']['entity']").alias('entity'),
    F.expr("cols['4']['resolved_text']").alias('idco_description'),
    F.expr("cols['5']").alias('icdo_code'),
).show(truncate=False)

+--------------------------+-----+---+------+-------------------------------------------+---------+
|chunk                     |begin|end|entity|idco_description                           |icdo_code|
+--------------------------+-----+---+------+-------------------------------------------+---------+
|Left breast adenocarcinoma|11   |36 |Cancer|Intraductal carcinoma, noninfiltrating, NOS|8500/2   |
|T3 N1b M0                 |44   |52 |Cancer|Kaposi sarcoma                             |9140/3   |
+--------------------------+-----+---+------+-------------------------------------------+---------+



Light Pipeline

In [10]:
light_result[0]['resolution']

[Annotation(entity, 11, 36, 8500/2, {'chunk': '0', 'all_k_results': '8500/2:::8520/3:::8520/2:::8470/2:::8200/3:::8312/3:::8420/3:::8140/3:::8190/3:::8522/3:::8318/3:::8290/3:::8470/3:::8250/2:::8140/2:::9110/3:::8260/3:::8571/3:::8317/3:::8220/3:::8144/3:::8252/3:::8251/3:::8210/3:::8253/2', 'all_k_distances': '0.8382:::1.1008:::1.1222:::1.1223:::1.1351:::1.1533:::1.2024:::1.2106:::1.2129:::1.2205:::1.2537:::1.2595:::1.2679:::1.2887:::1.2887:::1.2995:::1.3195:::1.3262:::1.3327:::1.3340:::1.3490:::1.3564:::1.3564:::1.3617:::1.3650', 'confidence': '0.0595', 'all_k_cosine_distances': '0.1997:::0.2287:::0.2297:::0.1907:::0.2311:::0.2119:::0.2368:::0.2917:::0.2647:::0.2271:::0.2269:::0.2722:::0.2636:::0.2791:::0.2791:::0.2520:::0.2584:::0.2947:::0.2546:::0.2499:::0.2993:::0.2558:::0.2558:::0.2705:::0.2544', 'all_k_resolutions': 'Intraductal carcinoma, noninfiltrating, NOS:::Lobular carcinoma, NOS:::Lobular carcinoma in situ:::Mucinous cystadenocarcinoma, non-invasive:::Adenoid cystic carci